In [1]:
!pip install denoiser
!pip install datasets
#!wget https://facebookresearch.github.io/denoiser/audio/noisy/alex_noisy.mp3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 186.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 816.3 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for denoiser: filename=denoiser-0.1.5-py3-none-any.whl size=43840 sha256=689ee2b0c463b4eb42c91cbb0aaefb8bfbbf8d88f6df495b1fb0afd6e06baab1
  Stored in directory: /root/.cache/pip/wheels/63/3e/0c/fd8cadf54cf581c5d21a374b112b8659cc0b5629d66c96cced
  Created wheel for pystoi: filename=pystoi-0.3.3-py2.py3-none-any.whl size=7793 sha256=ddf523ead41938d01339f31358e6c5b837aa5f3e94596b1328f6c744206cbd5b
  Stored in directory: /root/.cache/pip/wheels/46/4a/ad/3ab460193ed0535430b4b1575f255aa6bae69df17453628e86
  Created wheel for julius: filename=julius-0.2.7-py3-none-any.whl size=21894 sha256=ad2b3a3fc9ababcb9

In [2]:
from IPython import display as disp
import torch
import torchaudio
from denoiser import pretrained
from denoiser.dsp import convert_audio
import librosa 
import pandas as pd 
import numpy as np
from pathlib import Path
import shutil
import os
import glob
from tqdm import tqdm
from datasets import load_dataset,list_datasets,Audio


In [3]:
dset = load_dataset('openslr','SLR53')

Extracting data files:   0%|          | 0/16 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/218703 [00:00<?, ? examples/s]

Dataset open_slr downloaded and prepared to /root/.cache/huggingface/datasets/open_slr/SLR53/0.0.0/cd9de085f77465c1cb7d394e748a3a5b80fad2049e7af92f67baf04b894f9567. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
!mkdir /kaggle/tmp
os.chdir('/kaggle/tmp')
os.getcwd()

'/kaggle/tmp'

In [5]:
!git clone https://gitlab.com/mushrafi88/dlsprint.git

Cloning into 'dlsprint'...
remote: Enumerating objects: 197, done.
remote: Total 197 (delta 0), reused 0 (delta 0), pack-reused 197
Receiving objects: 100% (197/197), 38.48 MiB | 12.49 MiB/s, done.
Resolving deltas: 100% (79/79), done.
Updating files: 100% (75/75), done.


In [6]:
df_train_files_duration = pd.read_csv('/kaggle/tmp/dlsprint/csv_files/train_files_duration.csv')

In [7]:
df_train_files_duration

,path,duration
0,common_voice_bn_30614352.mp3,00:00:04
1,common_voice_bn_30614357.mp3,00:00:07
2,common_voice_bn_30614365.mp3,00:00:05
3,common_voice_bn_30646492.mp3,00:00:09
4,common_voice_bn_30646493.mp3,00:00:07
...,...,...
206945,common_voice_bn_31832728.mp3,00:00:10
206946,common_voice_bn_31832729.mp3,00:00:05
206947,common_voice_bn_31832730.mp3,00:00:09
206948,common_voice_bn_31832731.mp3,00:00:04


In [8]:
def get_all_unique_values(df):
    df_empty = {}
    for i in df.columns:
        df_empty[i] = df[i].unique()
    return df_empty

In [9]:
errors=["common_voice_bn_31727562",
        'common_voice_bn_30998934',
        'common_voice_bn_31595526',
        'common_voice_bn_31534853',
        'common_voice_bn_31518061',
        'common_voice_bn_31518373',
        'common_voice_bn_31613621',
        'common_voice_bn_31555333',
        'common_voice_bn_31772113',
        'common_voice_bn_31605391',
        'common_voice_bn_31631175',
        'common_voice_bn_31563901',
        'common_voice_bn_31691690',
        'common_voice_bn_31692010',
        'common_voice_bn_31683653',
        'common_voice_bn_31692182',
        'common_voice_bn_31519976',
        'common_voice_bn_31675793',
        'common_voice_bn_31019914',
        'common_voice_bn_31660287',
        'common_voice_bn_31660384',
        'common_voice_bn_31557261',
        'common_voice_bn_31633101',
        'common_voice_bn_31599243',
        'common_voice_bn_31521515',
        'common_voice_bn_31777802',
        'common_voice_bn_31777848',
        'common_voice_bn_31669646',
        'common_voice_bn_31566083',
        'common_voice_bn_31530331',
        'common_voice_bn_31727697',
        'common_voice_bn_31513270',
        'common_voice_bn_31686295',
        'common_voice_bn_31753693',
        'common_voice_bn_31686334',
        'common_voice_bn_31765546',
        'common_voice_bn_31765548',
        'common_voice_bn_31662742',
        'common_voice_bn_31704856',
        'common_voice_bn_31635344',
        'common_voice_bn_31618327',
        'common_voice_bn_31743074',
        'common_voice_bn_31678862',
        'common_voice_bn_31626674',
        'common_voice_bn_31626677',
        'common_voice_bn_31523889',
        'common_voice_bn_31610804',
        'common_voice_bn_31769538',
        'common_voice_bn_31533273',
        'common_voice_bn_31445621',
        'common_voice_bn_31620650']
errors = [i+'.mp3' for i in errors]

index_to_be_dropped_train = df_train_files_duration[df_train_files_duration['path'].isin(errors)].index
df_train_files_duration = df_train_files_duration.drop(index_to_be_dropped_train)

In [10]:
def audio_info_cleaning(df):
    df['duration'] = df['duration'].str.replace(':','0')
    df['duration'] = df['duration'].astype(int)
    return df

In [11]:
df_train_files_duration = audio_info_cleaning(df_train_files_duration)
df_train_files_duration['path'] = '/kaggle/input/dlsprint/train_files/' + df_train_files_duration['path']

df_train_files_duration

,path,duration
0,/kaggle/input/dlsprint/train_files/common_voic...,4
1,/kaggle/input/dlsprint/train_files/common_voic...,7
2,/kaggle/input/dlsprint/train_files/common_voic...,5
3,/kaggle/input/dlsprint/train_files/common_voic...,9
4,/kaggle/input/dlsprint/train_files/common_voic...,7
...,...,...
206945,/kaggle/input/dlsprint/train_files/common_voic...,10
206946,/kaggle/input/dlsprint/train_files/common_voic...,5
206947,/kaggle/input/dlsprint/train_files/common_voic...,9
206948,/kaggle/input/dlsprint/train_files/common_voic...,4


In [12]:
get_all_unique_values(df_train_files_duration)['duration']

array([ 4,  7,  5,  9,  6,  8,  2,  3, 10,  1, 11,  0])

In [13]:
df_train_files_duration[(df_train_files_duration['duration'] <=3) | (df_train_files_duration['duration'] >5)  ]

df_train_files_duration =df_train_files_duration.sample(10000)

df_train_files_duration['exist'] = df_train_files_duration['path'].apply(lambda x: os.path.exists(str(x)))

df_train_files_duration = df_train_files_duration[df_train_files_duration.exist == True]

openslr = dset['train']

openslr = openslr.cast_column("path", Audio(sampling_rate=16_000))

openslr

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 218703
})

In [14]:
openslr = openslr.train_test_split(0.05)

openslr=openslr['test']

df_openslr = openslr.to_pandas()

df_openslr=df_openslr.drop('audio',axis=1)

df_openslr

,path,sentence
0,"{'bytes': None, 'path': '/root/.cache/huggingf...",নির্দেশনা দেয় কেন্দ্রীয় ব্যাংক
1,"{'bytes': None, 'path': '/root/.cache/huggingf...",তিন লম্ফে
2,"{'bytes': None, 'path': '/root/.cache/huggingf...",হজরত মুহাম্মদ সা.
3,"{'bytes': None, 'path': '/root/.cache/huggingf...",তাদের কয়েকটিতে
4,"{'bytes': None, 'path': '/root/.cache/huggingf...",তাকে জনবিচ্ছিন্ন করব
...,...,...
10931,"{'bytes': None, 'path': '/root/.cache/huggingf...",রাজনৈতিক সমাবেশ
10932,"{'bytes': None, 'path': '/root/.cache/huggingf...",এমন সংসদ সদস্য
10933,"{'bytes': None, 'path': '/root/.cache/huggingf...",ব্লেড গেল
10934,"{'bytes': None, 'path': '/root/.cache/huggingf...",তাদের ছেলে-মেয়েদের


In [15]:
df_openslr['path'] =  df_openslr['path'].apply(lambda x:x['path'])
df_openslr

,path,sentence
0,/root/.cache/huggingface/datasets/downloads/ex...,নির্দেশনা দেয় কেন্দ্রীয় ব্যাংক
1,/root/.cache/huggingface/datasets/downloads/ex...,তিন লম্ফে
2,/root/.cache/huggingface/datasets/downloads/ex...,হজরত মুহাম্মদ সা.
3,/root/.cache/huggingface/datasets/downloads/ex...,তাদের কয়েকটিতে
4,/root/.cache/huggingface/datasets/downloads/ex...,তাকে জনবিচ্ছিন্ন করব
...,...,...
10931,/root/.cache/huggingface/datasets/downloads/ex...,রাজনৈতিক সমাবেশ
10932,/root/.cache/huggingface/datasets/downloads/ex...,এমন সংসদ সদস্য
10933,/root/.cache/huggingface/datasets/downloads/ex...,ব্লেড গেল
10934,/root/.cache/huggingface/datasets/downloads/ex...,তাদের ছেলে-মেয়েদের


In [16]:
openslr_files = df_openslr.path.to_list()

In [17]:
openslr_files[:5]

['/root/.cache/huggingface/datasets/downloads/extracted/dc97555a287fcaa38dd71eca32ec81b71bf8dda7e508cc7eaabf080fbe634172/asr_bengali/data/30/30c522fac3.flac',
 '/root/.cache/huggingface/datasets/downloads/extracted/4377f392f6b9d896dcb0c29e34d358687865640adb82da5f8553d159ee97c8cf/asr_bengali/data/97/97df4da7cb.flac',
 '/root/.cache/huggingface/datasets/downloads/extracted/562fa9a809fc2591996eb13678d50a726a67f159a5589392583217416f544fa9/asr_bengali/data/26/26621a8307.flac',
 '/root/.cache/huggingface/datasets/downloads/extracted/dc97555a287fcaa38dd71eca32ec81b71bf8dda7e508cc7eaabf080fbe634172/asr_bengali/data/3a/3aabb32dab.flac',
 '/root/.cache/huggingface/datasets/downloads/extracted/ed0ea3520aaa0d1f966696e7ec0927d93a3e1bfe7a8eac7890837421ca4c079c/asr_bengali/data/79/79d626b733.flac']

In [18]:
df_openslr['path'] = df_openslr['path'].apply(lambda x : os.path.basename(str(x)))

In [19]:
df_openslr

,path,sentence
0,30c522fac3.flac,নির্দেশনা দেয় কেন্দ্রীয় ব্যাংক
1,97df4da7cb.flac,তিন লম্ফে
2,26621a8307.flac,হজরত মুহাম্মদ সা.
3,3aabb32dab.flac,তাদের কয়েকটিতে
4,79d626b733.flac,তাকে জনবিচ্ছিন্ন করব
...,...,...
10931,056711c865.flac,রাজনৈতিক সমাবেশ
10932,a3230c3684.flac,এমন সংসদ সদস্য
10933,7f87c122cb.flac,ব্লেড গেল
10934,a6463ecbb0.flac,তাদের ছেলে-মেয়েদের


In [20]:
train_files = df_train_files_duration['path'].to_list()
train_files[:5]

['/kaggle/input/dlsprint/train_files/common_voice_bn_31786865.mp3',
 '/kaggle/input/dlsprint/train_files/common_voice_bn_31546487.mp3',
 '/kaggle/input/dlsprint/train_files/common_voice_bn_31693875.mp3',
 '/kaggle/input/dlsprint/train_files/common_voice_bn_31630968.mp3',
 '/kaggle/input/dlsprint/train_files/common_voice_bn_31786480.mp3']

In [21]:
df_train_files = pd.DataFrame()
df_train_files['path'] = train_files
df_train_files['path'] = df_train_files['path'].apply(lambda x : os.path.basename(str(x)))

In [22]:
df_train_files

,path
0,common_voice_bn_31786865.mp3
1,common_voice_bn_31546487.mp3
2,common_voice_bn_31693875.mp3
3,common_voice_bn_31630968.mp3
4,common_voice_bn_31786480.mp3
...,...
9995,common_voice_bn_31755897.mp3
9996,common_voice_bn_31697476.mp3
9997,common_voice_bn_31508854.mp3
9998,common_voice_bn_31623563.mp3


In [23]:
!mkdir -p /kaggle/working/denoised/{validation_files,train_files,slr53}
!mkdir -p /kaggle/working/raw/{train_files,slr53}

In [24]:
for files in tqdm(openslr_files):
    shutil.copy2(files, '/kaggle/working/raw/slr53/')

100%|██████████| 10936/10936 [00:20<00:00, 525.80it/s]


In [25]:
for files in tqdm(train_files):
    shutil.copy2(files,'/kaggle/working/raw/train_files/')

100%|██████████| 10000/10000 [00:50<00:00, 198.30it/s]


In [26]:
df_train_files.to_csv('/kaggle/working/df_train_files_10k.csv')

In [27]:
df_openslr.to_csv('/kaggle/working/df_openslr.csv')

In [28]:
def denoiser_fb(file_name):
    model = pretrained.dns64().cuda()
    wav, sr = torchaudio.load(file_name)
    wav = convert_audio(wav.cuda(), sr, model.sample_rate, model.chin)
    with torch.no_grad():
        denoised = model(wav[None])[0]
    return denoised.data.cpu(),model.sample_rate

In [29]:
def denoised2mp3(list_of_songs,directory_to_be_stored):
    for file_name in tqdm(list_of_songs):
        file_output = 'denoised_' + os.path.basename(file_name).split('/')[-1][:-4] + '.mp3'
        array,sample_rate = denoiser_fb(file_name)
        path_to_save = directory_to_be_stored + '/'+ file_output
        torchaudio.save(path_to_save, array, sample_rate)

In [30]:
audio_path  = '/kaggle/input/dlsprint/validation_files'

directory_to_be_stored = '/kaggle/working/denoised/validation_files'

list_of_songs = glob.glob(f"{audio_path}/*.mp3")

denoised2mp3(list_of_songs,directory_to_be_stored)

  0%|          | 0/7747 [00:00<?, ?it/s]Downloading: "https://dl.fbaipublicfiles.com/adiyoss/denoiser/dns64-a7761ff99a7d5bb6.th" to /root/.cache/torch/hub/checkpoints/dns64-a7761ff99a7d5bb6.th


  0%|          | 0.00/128M [00:00<?, ?B/s]

100%|██████████| 7747/7747 [1:24:10<00:00,  1.53it/s]


In [31]:
#train files
directory_to_be_stored = '/kaggle/working/denoised/train_files'

denoised2mp3(train_files,directory_to_be_stored)

100%|██████████| 10000/10000 [1:49:12<00:00,  1.53it/s]


In [32]:

directory_to_be_stored = '/kaggle/working/denoised/slr53'

denoised2mp3(openslr_files ,directory_to_be_stored)

100%|██████████| 10936/10936 [1:49:33<00:00,  1.66it/s]
